# Captura de dados - Batalha de Música
Etapa de captura de dados e estruturação para a Análise Exploratória

## Etapa 1: Instalando e importando bibliotecas

In [ ]:
# Instalando bibliotecas
! pip3 install spotipy
! pip3 install pandas

In [10]:
# Importando bibliotecas
import spotipy as sp
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import os

## Etapa 2: Autenticação da API e requisição

In [20]:
# Recebendo variáveis de ambiente com chave e segredo da API
CLIENT_ID_SPOTIFY = os.environ.get('CLIENT_ID_SPOTIFY')
CLIENT_SECRET_SPOTIFY = os.environ.get('CLIENT_SECRET_SPOTIFY')

In [21]:
# Autenticando credenciais
spotify = sp.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID_SPOTIFY, client_secret=CLIENT_SECRET_SPOTIFY))

# Id da Beyoncé e Taylor Swift no Spotify
ids = ['6vWDO969PvNqNYHIOW5v0m', #Bey
       '06HL4z0CvFAxyc27GXpf02', #Taylor Swift
       ]

# DataFrame vazio para armazenar as informações das faixas
df_list = []

In [ ]:
# Realizando requisições no Spotify
for artist_id in ids:
    # Obter o nome do artista
    artist_info = spotify.artist(artist_id)
    artist_name = artist_info['name']
    artist_image_url = artist_info['images'][0]['url'] if artist_info['images'] else None  # Pega a primeira imagem, se disponível

    # Obter os álbuns do artista usando a API do Spotify
    albums = spotify.artist_albums(artist_id, album_type='album,single', country='BR', limit=50)

    # Processar cada álbum e obter todas as faixas únicas
    for album in albums['items']:
        album_tracks = spotify.album_tracks(album['id'])

        for track in album_tracks['items']:
            track_id = track['id']
            track_info = spotify.track(track_id)
            audio_features = spotify.audio_features(track_id)[0]
            audio_analysis = spotify.audio_analysis(track_id)

            track_name = track_info['name']
            duration_ms = audio_features['duration_ms']
            energy = audio_features['energy']
            tempo = audio_features['tempo']
            loudness = audio_features['loudness']
            release_date = track_info['album']['release_date']
            album_name = track_info['album']['name']
            album_type = track_info['album']['type']
            popularity = track_info['popularity']
            
            #Novo para Vini
            acousticness = audio_features['acousticness']
            danceability = audio_features['danceability']
            instrumentalness = audio_features['instrumentalness']
            liveness = audio_features['liveness']
            speechiness = audio_features['speechiness']
            valence = audio_features['valence']

            # Extração de informações do objeto 'track' em 'audio_analysis'
            num_samples = audio_analysis['track']['num_samples']
            duration = audio_analysis['track']['duration']
            loudness_analysis = audio_analysis['track']['loudness']
            tempo_analysis = audio_analysis['track']['tempo']

            track_data = {
                'Artist': artist_name,
                'ArtistImageURL': artist_image_url,
                'Track': track_name,
                'Popularity': popularity,
                'Duration_ms': duration_ms,
                'Album Name': album_name,
                'Album Type': album_type,
                'acousticness': acousticness,
                'danceability': danceability,
                'instrumentalness': instrumentalness,
                'liveness': liveness,
                'speechiness': speechiness,
                'valence': valence,
                'Energia': energy,
                'Tempo': tempo,
                'Loudness': loudness,
                'Num_samples': num_samples,
                'Duration_analysis': duration,
                'Loudness_analysis': loudness_analysis,
                'Tempo_analysis': tempo_analysis,
                'Release_date_precision': release_date,
                'Id': track_id
            }

            df_list.append(track_data)

# Criar o DataFrame
df = pd.DataFrame(df_list)
df.head()

## Etapa 3: Exportando arquivo

In [ ]:
# Exportando arquivo para análise
df.to_csv('/content/sample_data/bey_tay_2023_09_13.csv', index=False)